In [6]:
import os
import rasterio
from rasterio.merge import merge
import glob
import cProfile
import io
import os
import requests
import pstats

import zipfile
import numpy as np

import functions

# FOR THE POINT CLOUDS

In [14]:
def download_and_extract(url, file_path, output_folder):
    """Download a ZIP file from a URL, extract its contents, and delete the ZIP file."""
    if not os.path.exists(file_path):
        try:
            print(f"Downloading from {url}...")
            response = requests.get(url, verify=False)  # Bypass SSL verification
            response.raise_for_status()  # Raise an error for bad responses
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded and saved to {file_path}")

            # Extract the ZIP file
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(output_folder)  # Extract to the output folder
            print(f"Extracted {file_path}")

            # Remove the ZIP file after extraction
            os.remove(file_path)
            print(f"Deleted the ZIP file: {file_path}")

        except requests.exceptions.RequestException as e:
            print(f"Failed to download {url}: {e}")
        except zipfile.BadZipFile as e:
            print(f"Failed to extract {file_path}: {e}")
        except Exception as e:
            print(f"An error occurred: {e}")

def download_raster_tiles(tile_list_file, output_folder):
    """
    Download DSM and DTM files for each tile specified in a text file.
    ------
    Input:
    - tile_list_file (str): Path to the text file containing the list of tiles to download.
    - output_folder (str): Directory where the downloaded and unzipped files will be saved.
    Output:
    - none:  The function writes output files directly to the specified `output_folder`.
    """
    
    # Base URLs for downloading DTM and DSM tiles
    base_url_dtm = "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/02a_DTM_0.5m"
    base_url_dsm = "https://ns_hwh.fundaments.nl/hwh-ahn/ahn4/03a_DSM_0.5m"
    
    # Read tile names from the text file
    with open(tile_list_file, 'r') as f:
        tile_names = [line.strip() for line in f.readlines() if line.strip()]

    os.makedirs(output_folder, exist_ok=True)
    
    for tile_name in tile_names:
        # Construct the URLs for DTM and DSM
        dtm_url = f"{base_url_dtm}/M_{tile_name}.zip"
        dsm_url = f"{base_url_dsm}/R_{tile_name}.zip"
        
        # Define file paths for downloaded zip files
        dtm_file_path = os.path.join(output_folder, f"M_{tile_name}.zip")
        dsm_file_path = os.path.join(output_folder, f"R_{tile_name}.zip")
        
        # Download and extract DTM and DSM files
        download_and_extract(dtm_url, dtm_file_path, output_folder)
        download_and_extract(dsm_url, dsm_file_path, output_folder)

In [15]:
download_raster_tiles("ams_tiles.txt", "../data")

C:\Users\jessi\anaconda3\envs\synthesis\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ns_hwh.fundaments.nl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded and saved to ../data\M_25BZ1.zip
Extracted ../data\M_25BZ1.zip
Deleted the ZIP file: ../data\M_25BZ1.zip


C:\Users\jessi\anaconda3\envs\synthesis\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ns_hwh.fundaments.nl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


KeyboardInterrupt: 

# for the dsm and dtm

In [ ]:
def download_files(tile_list_file):
    """
    Download LAZ files for each tile specified in a text file.
    
    Input:
    - tile_list_file (str): Path to the text file containing the list of tiles to download.
    """

    base_url = "https://geotiles.citg.tudelft.nl/AHN4_T"

    # Read tile names from the text file
    with open(tile_list_file, 'r') as f:
        tile_names = [line.strip() for line in f.readlines() if line.strip()]

    for full_tile_name in tile_names:
        if '_' in full_tile_name:
            tile_name, sub_tile = full_tile_name.split('_')
        else:
            print(f"Skipping invalid tile entry: {full_tile_name}")
            continue

        # Define folder structure and ensure it exists
        folder = f"../data/{tile_name}"
        os.makedirs(folder, exist_ok=True)

        # Construct the URL and file path
        sub_tile_str = f"_{int(sub_tile):02}"  # Convert sub_tile to zero-padded two-digit number
        url = f"{base_url}/{tile_name}{sub_tile_str}.LAZ"
        filename = f"{tile_name}{sub_tile_str}.LAZ"
        file_path = os.path.join(folder, filename)

        # Check if file already exists to avoid re-downloading
        if os.path.exists(file_path):
            print(f"File {file_path} already exists, skipping download.")
            continue

        # Attempt to download the file
        try:
            response = requests.get(url)
            response.raise_for_status()
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded and saved {file_path}")
        except requests.exceptions.RequestException as e:
            print(f"Failed to download {url}: {e}")

In [2]:
def merge_tif_files(input_folder, output_file, nodata_value=-9999):
    tif_files = glob.glob(os.path.join(input_folder, '*.TIF'))

    if not tif_files:
        print("No TIF files found.")
        return

    src_files_to_mosaic = []

    for tif_file in tif_files:
        src = rasterio.open(tif_file)
        src_files_to_mosaic.append(src)

    mosaic, out_transform = merge(src_files_to_mosaic)

    # Replace existing nodata values with new nodata value if necessary
    for src in src_files_to_mosaic:
        if 'nodata' in src.meta:
            mosaic[mosaic == src.nodata] = nodata_value

    out_meta = src_files_to_mosaic[0].meta.copy()

    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_transform,
        "count": mosaic.shape[0],  # Band count
        "nodata": nodata_value  # Set the nodata value
    })

    # Write the merged file to the output path
    with rasterio.open(output_file, "w", **out_meta) as dest:
        dest.write(mosaic)

    print(f"Merged {len(tif_files)} TIF files into {output_file}")

In [3]:
input_folder = "data/DSM"
output_file = "data/DSM_ams.TIF"

In [4]:
merge_tif_files(input_folder, output_file)

Merged 14 TIF files into data/DSM_ams.TIF


In [6]:
input_folder = "data/DTM"
output_file = "data/DTM_ams.TIF"
merge_tif_files(input_folder, output_file)

Merged 14 TIF files into data/DTM_ams.TIF


In [ ]:
import os
import glob
import rasterio
from rasterio.merge import merge

def merge_tif_files(tif_files, output_file, nodata_value=-9999):
    """Merge specified TIF files and save as a single output file."""
    
    if not tif_files:
        print("No TIF files found.")
        return

    src_files_to_mosaic = []

    for tif_file in tif_files:
        src = rasterio.open(tif_file)
        src_files_to_mosaic.append(src)

    mosaic, out_transform = merge(src_files_to_mosaic)

    # Replace existing nodata values with new nodata value if necessary
    for src in src_files_to_mosaic:
        if 'nodata' in src.meta:
            mosaic[mosaic == src.nodata] = nodata_value

    out_meta = src_files_to_mosaic[0].meta.copy()

    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_transform,
        "count": mosaic.shape[0],  
        "nodata": nodata_value  
    })

    # Write the merged file to the output path
    with rasterio.open(output_file, "w", **out_meta) as dest:
        dest.write(mosaic)

    print(f"Merged {len(tif_files)} TIF files into {output_file}")

    # Clean up: delete the original files
    for src in src_files_to_mosaic:
        src.close()  # Close the dataset
        os.remove(src.name)  # Delete the original TIF file
        print(f"Deleted file: {src.name}")

def merge_and_cleanup(input_folder, output_dtm_file, output_dsm_file):
    """Merge M_ and R_ TIF files and clean up the original files.
    ------
    input: 
    - input_folder (string):    path to folder containing the seperate .tif files.
    - output_dtm_file (string): path to output dtm file
    output:"""
    # Collect TIF files in the input folder
    all_tif_files = glob.glob(os.path.join(input_folder, '*.TIF'))

    # Filter M_ and R_ TIF files
    m_tif_files = [f for f in all_tif_files if os.path.basename(f).startswith('M_')]
    r_tif_files = [f for f in all_tif_files if os.path.basename(f).startswith('R_')]

    # Merge M_ files
    print("Merging M_ TIF files...")
    merge_tif_files(m_tif_files, output_m_file)

    # Merge R_ files
    print("Merging R_ TIF files...")
    merge_tif_files(r_tif_files, output_r_file)